In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("FeatureEngineering").master("local[*]").config("spark.submit.pyFiles", "../holidays_package.zip").getOrCreate()

25/05/28 09:16:06 WARN Utils: Your hostname, watanaberyounoMacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.16.164 instead (on interface en0)
25/05/28 09:16:06 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
25/05/28 09:16:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
----------------------------------------
Exception occurred during processing of request from ('127.0.0.1', 65095)
Traceback (most recent call last):
  File "/Users/ryo/.pyenv/versions/3.10.8/lib/python3.10/socketserver.py", line 316, in _handle_request_noblock
    self.process_request(request, client_address)
  File "/Users/ryo/.pyenv/versions/3.10.8/lib/python3.10/socketserver.py", line 347, in process_request
    self.finish_request(request, client_address)
  File

# Data_LoaderのPySpark検証

In [2]:
import os
import zipfile

import pandas as pd

# ZIPファイルを展開
zip_dir = "../data/power_usage"
extract_dir = "../data/power_extracted"
os.makedirs(extract_dir, exist_ok=True)

for zip_name in sorted(os.listdir(zip_dir)):
    if zip_name.endswith(".zip"):
        zip_path = os.path.join(zip_dir, zip_name)
        with zipfile.ZipFile(zip_path, "r") as zip_ref:
            zip_ref.extractall(extract_dir)

In [3]:
from pyspark.sql.functions import col, to_date
from pyspark.sql.types import IntegerType, LongType, StringType, StructField, StructType

In [4]:
def read_and_extract_max_power(csv_path):
    try:
        df = pd.read_csv(csv_path, encoding="shift-jis", skiprows=54)
        max_power = int(df["当日実績(５分間隔値)(万kW)"].max())
        date = os.path.basename(csv_path).split("_")[0]  # YYYYMMDDを取得
        return {"date": date, "max_power": max_power}
    except Exception as e:
        print(f"Error in {csv_path}: {e}")
        return None

# pandasで抽出し、Spark DataFrame化
records = []
for fname in os.listdir(extract_dir):
    if fname.endswith(".csv"):
        path = os.path.join(extract_dir, fname)
        record = read_and_extract_max_power(path)
        if record:
            records.append(record)

# pandas → Spark DataFrame
power_usage_df = spark.createDataFrame(records, StructType([
    StructField("date", StringType(), True),
    StructField("max_power", LongType(), True),
]))

# 日付形式に変換
power_usage_df = power_usage_df.withColumn("date", to_date(col("date"), "yyyyMMdd"))

# 表示確認
power_usage_df.show(5)

+----------+---------+
|      date|max_power|
+----------+---------+
|2023-02-19|     3216|
|2022-10-03|     3813|
|2023-11-12|     3160|
|2023-02-24|     4110|
|2024-02-07|     4791|
+----------+---------+
only showing top 5 rows



In [5]:
def load_weather_data(path) -> pd.DataFrame:
        """気象データファイルを読み込む
        pysparkだとskiprowが設定できないため一度pandas dataframeで読み込む

        Args:
            encoding: ファイルエンコーディング
            skiprows: スキップする行番号のリスト

        Returns:
            pd.DataFrame: 気象データフレーム
        """
        df = pd.read_csv(path, encoding="shift-jis", skiprows=[0, 1, 2, 4, 5])

        # 必要なカラムだけ抽出
        df = df[["年月日", "最高気温(℃)", "最低気温(℃)", "天気概況(昼：06時〜18時)"]]

        # カラム名を英語に変更
        df = df.rename(
            columns={
                "年月日": "date",
                "最高気温(℃)": "max_temp",
                "最低気温(℃)": "min_temp",
                "天気概況(昼：06時〜18時)": "weather",
            },
        )# 日付をdatetime型に変換
        df["date"] = pd.to_datetime(df["date"], format="%Y/%m/%d")

        return df

In [6]:
path = "../data/weather_data.csv"
df = load_weather_data(path)

In [7]:
weather_df = spark.createDataFrame(df)
# timestampからdateにするためSparkに渡したあとに明示的に日付だけに変換
weather_df = weather_df.withColumn("date", to_date("date"))

In [8]:
weather_df.show(5)

+----------+--------+--------+--------+
|      date|max_temp|min_temp| weather|
+----------+--------+--------+--------+
|2018-01-01|    13.0|     0.4|      晴|
|2018-01-02|    10.8|     0.8|    快晴|
|2018-01-03|     8.6|     2.3|    快晴|
|2018-01-04|     9.6|     0.0|    快晴|
|2018-01-05|     6.3|     0.8|曇一時雨|
+----------+--------+--------+--------+
only showing top 5 rows



In [9]:
# データの結合
weather_df = weather_df.repartition(60, "date")
power_usage_df = power_usage_df.repartition(60, "date")

merge_data = weather_df.join(
    power_usage_df,
    on = ["date"],
    how="inner",
)

In [10]:
merge_data.rdd.getNumPartitions()

200

# 前処理、特徴量エンジニアリングの確認

### 天気のカテゴリ変数まとめ

In [11]:
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType


@udf(returnType=StringType())
def categorize_weather(weather):
    if weather is None:
        return "不明"
    if "雪" in weather or "ゆき" in weather:
        return "雪"
    if "雷" in weather:
        if "雨" in weather or "あめ" in weather:
            return "雷雨"
        if "晴" in weather:
            return "晴れ(雷あり)"
        if "曇" in weather:
            return "曇り(雷あり)"
        return "雷"
    if "快晴" in weather:
        return "快晴"
    if "晴" in weather:
        if "曇" in weather:
            return "晴れ時々曇り"
        if "雨" in weather or "あめ" in weather or "雷" in weather:
            return "晴れ時々雨"
        return "晴れ"
    if "曇" in weather:
        if "雨" in weather or "あめ" in weather:
            return "曇り時々雨"
        return "曇り"
    if "雨" in weather or "あめ" in weather:
        return "雨"
    return "その他"

In [12]:
def categorize_weather_spark(df, weather_col="weather"):
    return df.withColumn("weather_category", categorize_weather(df[weather_col])).drop(weather_col)

In [13]:
temp = categorize_weather_spark(merge_data)

In [14]:
temp.show()

+----------+--------+--------+---------+----------------+
|      date|max_temp|min_temp|max_power|weather_category|
+----------+--------+--------+---------+----------------+
|2022-07-31|    35.0|    27.2|     4691|            晴れ|
|2023-06-22|    22.6|    16.7|     3456|      曇り時々雨|
|2023-07-15|    32.9|    26.5|     3861|            曇り|
|2024-09-18|    35.1|    24.8|     5424|    晴れ時々曇り|
|2022-07-27|    33.6|    26.2|     5256|    晴れ時々曇り|
|2022-08-02|    35.9|    27.6|     5952|    晴れ時々曇り|
|2022-11-29|    21.3|    11.6|     3572|      曇り時々雨|
|2022-12-25|    13.1|     1.8|     3741|            晴れ|
|2023-05-22|    29.4|    16.5|     3717|    晴れ時々曇り|
|2023-09-14|    32.7|    25.1|     5036|            晴れ|
|2023-11-08|    21.6|    14.6|     3223|            快晴|
|2024-02-05|     6.2|     0.3|     5007|              雪|
|2024-05-30|    27.3|    17.5|     3542|    晴れ時々曇り|
|2024-06-04|    25.5|    14.8|     3470|            晴れ|
|2024-06-12|    30.1|    20.0|     4041|    晴れ時々曇り|
|2024-08-27|   

### 数値系特徴量作成

In [15]:
def create_numeric_features_spark(df):
    avg = (col("max_temp") + col("min_temp")) / 2
    rng = col("max_temp") - col("min_temp")
    cdd = (avg - 18).cast("double")
    hdd = (18 - avg).cast("double")

    return (
        df.withColumn("avg", avg)
          .withColumn("rng", rng)
          .withColumn("cdd", cdd)
          .withColumn("hdd", hdd)
          .withColumn("hot", (col("max_temp") >= 30))
          .withColumn("cold", (col("min_temp") <= 5))
    )

In [16]:
temp = create_numeric_features_spark(temp)

### カレンダー系特徴量作成

In [17]:
# spark.stop()

# spark = SparkSession.builder \
#     .appName("holiday-udf") \
#     .master("local[*]") \
#     .config("spark.submit.pyFiles", "../holidays_package.zip") \
#     .getOrCreate()

In [18]:
import holidays
import numpy as np
from pyspark.sql.functions import cos, dayofmonth, dayofweek, month, sin, year

jp_holidays = holidays.Japan()

# 祝日判定
@udf(returnType=IntegerType())
def is_holiday(date):
    return int(date in jp_holidays)

def create_calendar_features_spark(df, date_col="date"):
    return (
        df.withColumn("year", year(col(date_col)))
          .withColumn("month", month(col(date_col)))
          .withColumn("day", dayofmonth(col(date_col)))
        # 1=日曜〜7=土曜
          .withColumn("dow", dayofweek(col(date_col)))
          .withColumn("dow_sin", sin(2 * np.pi * (col("dow") - 1) / 7))
          .withColumn("dow_cos", cos(2 * np.pi * (col("dow") - 1) / 7))
          .withColumn("mon_sin", sin(2 * np.pi * col("month") / 12))
          .withColumn("mon_cos", cos(2 * np.pi * col("month") / 12))
          .withColumn("weekend", ((col("dow") == 1) | (col("dow") == 7)).cast("int"))
          .withColumn("holiday", is_holiday(col(date_col)))
    )

In [19]:
test = create_calendar_features_spark(temp)

In [20]:
test.show(5)

+----------+--------+--------+---------+----------------+------------------+-----------------+------------------+-------------------+-----+-----+----+-----+---+---+-------------------+-------------------+--------------------+--------------------+-------+-------+
|      date|max_temp|min_temp|max_power|weather_category|               avg|              rng|               cdd|                hdd|  hot| cold|year|month|day|dow|            dow_sin|            dow_cos|             mon_sin|             mon_cos|weekend|holiday|
+----------+--------+--------+---------+----------------+------------------+-----------------+------------------+-------------------+-----+-----+----+-----+---+---+-------------------+-------------------+--------------------+--------------------+-------+-------+
|2022-07-31|    35.0|    27.2|     4691|            晴れ|              31.1|7.800000000000001|13.100000000000001|-13.100000000000001| true|false|2022|    7| 31|  1|                0.0|                1.0| -0.49999